In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

// Style laden (optional)
try {
  const css = readFileSync("../style.css", "utf8");
  display.html(`<style>${css}</style>`);
} catch (e) {}

// Imports
import { buildParser } from '@lezer/generator';
import { Tree, TreeCursor } from '@lezer/common';
import { LRParser } from '@lezer/lr';
// Space Instruction: RecursiveSet verwenden
import { RecursiveSet, Tuple } from "./recursive-set";

// ============================================================================
// 1. AST Types (Tuples with Value Semantics)
// ============================================================================

/**
 * Base class for all Expression Nodes.
 * Inherits from Tuple to ensure structural equality (e.g., x+1 === x+1).
 */
abstract class ExprNode extends Tuple<any[]> {
    /** Returns operator precedence for pretty printing parentheses */
    abstract precedence(): number;
}

// --- Atomic Values ---

class Num extends ExprNode {
    constructor(val: number) { super("num", val); }
    get value(): number { return this.get(1); }
    precedence() { return 4; }
    toString() { return String(this.value); }
}

class VarX extends ExprNode {
    constructor() { super("x"); }
    precedence() { return 4; }
    toString() { return "x"; }
}

// --- Unary Operations ---

class ExpFunc extends ExprNode {
    constructor(arg: ExprNode) { super("exp", arg); }
    get arg(): ExprNode { return this.get(1); }
    precedence() { return 4; }
    toString() { return `exp(${this.arg})`; }
}

class LnFunc extends ExprNode {
    constructor(arg: ExprNode) { super("ln", arg); }
    get arg(): ExprNode { return this.get(1); }
    precedence() { return 4; }
    toString() { return `ln(${this.arg})`; }
}

// --- Binary Operations ---

abstract class BinOp extends ExprNode {
    constructor(op: string, left: ExprNode, right: ExprNode) { 
        super(op, left, right); 
    }
    get op(): string { return this.get(0); }
    get left(): ExprNode { return this.get(1); }
    get right(): ExprNode { return this.get(2); }
    
    // Pretty Printing with minimal parentheses logic
    toString() {
        const p = this.precedence();
        let l = this.left.toString();
        let r = this.right.toString();
        
        // Add parens if left child has lower precedence
        if (this.left.precedence() < p) l = `(${l})`;
        
        // Right-associativity check for Power ('**') vs Left-assoc for others
        if (this.op === "**") {
            // Right-associative: a ** b ** c -> a ** (b ** c)
            // Parenthesize right only if strictly lower
            if (this.right.precedence() < p) r = `(${r})`;
        } else {
            // Left-associative: a - b - c -> (a - b) - c
            // Parenthesize right if lower OR EQUAL
            if (this.right.precedence() <= p) r = `(${r})`;
        }
        return `${l} ${this.op} ${r}`;
    }
}

class Add extends BinOp { constructor(l: ExprNode, r: ExprNode) { super("+", l, r); } precedence() { return 1; } }
class Sub extends BinOp { constructor(l: ExprNode, r: ExprNode) { super("-", l, r); } precedence() { return 1; } }
class Mul extends BinOp { constructor(l: ExprNode, r: ExprNode) { super("*", l, r); } precedence() { return 2; } }
class Div extends BinOp { constructor(l: ExprNode, r: ExprNode) { super("/", l, r); } precedence() { return 2; } }
class Pow extends BinOp { constructor(l: ExprNode, r: ExprNode) { super("**", l, r); } precedence() { return 3; } }

// ============================================================================
// 2. Grammar (Lezer Definition)
// ============================================================================

const grammarDefinition = `
@top Expression { expr }

expr {
  Product ((Plus | Minus) Product)*
}

Product {
  Factor ((Mul | Div) Factor)*
}

Factor {
  // Right-recursive structure for power: Base (** Factor)?
  PowerExpr { Base (Power Factor)? } |
  Base
}

Base {
  ExpFunc { Exp "(" expr ")" } |
  LnFunc { Ln "(" expr ")" } |
  ParenExpr { "(" expr ")" } |
  Number |
  VarX
}

@tokens {
  Number { "0" | $[1-9] $[0-9]* }
  VarX { "x" }
  Exp { "exp" }
  Ln { "ln" }
  Power { "**" }
  Plus { "+" }
  Minus { "-" }
  Mul { "*" }
  Div { "/" }
  space { $[ \\t\\n\\r]+ }
  "(" ")"
}

@skip { space }
`;

const parser: LRParser = buildParser(grammarDefinition);

// ============================================================================
// 3. AST Transformation (Lezer Tree -> Tuple AST)
// ============================================================================

function lezerToAST(cursor: TreeCursor, source: string): ExprNode {
    const name = cursor.name;
    const text = source.slice(cursor.from, cursor.to);

    if (name === "⚠") throw new Error(`Syntax Error near: '${text}'`);

    switch (name) {
        case "Expression": // Wrapper
            cursor.firstChild();
            const res = lezerToAST(cursor, source);
            cursor.parent();
            return res;

        case "expr":
            return parseLeftAssociative(cursor, source, ["+", "-"]);
        
        case "Product":
            return parseLeftAssociative(cursor, source, ["*", "/"]);

        case "PowerExpr": {
            // Structure: Base (** Factor)?
            cursor.firstChild(); // Enter Base
            const base = lezerToAST(cursor, source);
            
            if (cursor.nextSibling()) { // Check if we have "**"
                // Skip "**" token, move to Factor
                cursor.nextSibling(); 
                const exponent = lezerToAST(cursor, source);
                cursor.parent();
                return new Pow(base, exponent);
            }
            cursor.parent();
            return base;
        }

        case "ExpFunc": {
            cursor.firstChild(); // "exp"
            cursor.nextSibling(); // "("
            cursor.nextSibling(); // expr
            const arg = lezerToAST(cursor, source);
            cursor.parent();
            return new ExpFunc(arg);
        }

        case "LnFunc": {
            cursor.firstChild(); // "ln"
            cursor.nextSibling(); // "("
            cursor.nextSibling(); // expr
            const arg = lezerToAST(cursor, source);
            cursor.parent();
            return new LnFunc(arg);
        }

        case "ParenExpr": {
            cursor.firstChild(); // "("
            cursor.nextSibling(); // expr
            const inner = lezerToAST(cursor, source);
            cursor.parent();
            return inner;
        }

        case "Number":
            return new Num(parseInt(text));

        case "VarX":
            return new VarX();

        // Fallback for pass-through nodes (e.g. Factor -> Base)
        default:
            if (cursor.firstChild()) {
                const child = lezerToAST(cursor, source);
                cursor.parent();
                return child;
            }
            throw new Error(`Unknown AST Node: ${name}`);
    }
}

/**
 * Handles Left-Associativity for +,-,*,/
 */
function parseLeftAssociative(cursor: TreeCursor, source: string, ops: string[]): ExprNode {
    const operands: ExprNode[] = [];
    const operators: string[] = [];

    if (cursor.firstChild()) {
        do {
            const t = source.slice(cursor.from, cursor.to);
            if (ops.includes(t)) {
                operators.push(t);
            } else {
                operands.push(lezerToAST(cursor, source));
            }
        } while (cursor.nextSibling());
        cursor.parent();
    }

    let result = operands[0];
    for (let i = 0; i < operators.length; i++) {
        const op = operators[i];
        const right = operands[i+1];
        
        switch(op) {
            case "+": result = new Add(result, right); break;
            case "-": result = new Sub(result, right); break;
            case "*": result = new Mul(result, right); break;
            case "/": result = new Div(result, right); break;
        }
    }
    return result;
}

function parseExpr(s: string): ExprNode {
    const tree = parser.parse(s);
    return lezerToAST(tree.cursor(), s);
}

// ============================================================================
// 4. Symbolic Differentiation (Switch-Case Implementation)
// ============================================================================

function diff(e: ExprNode): ExprNode {
    // We switch on the first element of the Tuple (the tag/kind)
    const tag = e.get(0);

    switch (tag) {
        case "num": // Constant c -> 0
            return new Num(0);
        
        case "x": // Variable x -> 1
            return new Num(1);

        case "+": { // u + v -> u' + v'
            const node = e as Add;
            return new Add(diff(node.left), diff(node.right));
        }

        case "-": { // u - v -> u' - v'
            const node = e as Sub;
            return new Sub(diff(node.left), diff(node.right));
        }

        case "*": { // u * v -> u'v + uv' (Product Rule)
            const node = e as Mul;
            return new Add(
                new Mul(diff(node.left), node.right),
                new Mul(node.left, diff(node.right))
            );
        }

        case "/": { // u / v -> (u'v - uv') / v^2 (Quotient Rule)
            const node = e as Div;
            return new Div(
                new Sub(
                    new Mul(diff(node.left), node.right),
                    new Mul(node.left, diff(node.right))
                ),
                new Mul(node.right, node.right)
            );
        }

        case "ln": { // ln(u) -> u' / u
            const node = e as LnFunc;
            return new Div(diff(node.arg), node.arg);
        }

        case "exp": { // exp(u) -> u' * exp(u)
            const node = e as ExpFunc;
            return new Mul(diff(node.arg), node);
        }

        case "**": { // u^v -> exp(v * ln(u)) -> Chain Rule
            const node = e as Pow;
            // Rewrite rule: D(u^v) = D( exp(v * ln(u)) )
            // We construct the AST for the rewritten form and differentiate that.
            const rewritten = new ExpFunc(
                new Mul(node.right, new LnFunc(node.left))
            );
            return diff(rewritten);
        }

        default:
            throw new Error(`Differentiation not implemented for: ${tag}`);
    }
}

// ============================================================================
// 5. Main Interface & Tests
// ============================================================================

function diffString(s: string): string {
    const t = parseExpr(s);
    const d = diff(t);
    return d.toString();
}

function test(s: string): void {
    console.log(`Input:   ${s}`);
    try {
        const parsed = parseExpr(s);
        // Note: parsed.toString() is automatic pretty printing from Tuple classes
        console.log(`Parsed:  ${parsed}`); 
        
        const derivative = diffString(s);
        console.log(`Deriv:   ${derivative}`);
    } catch (e) {
        console.error("Error:", e instanceof Error ? e.message : e);
    }
    console.log("-".repeat(50));
}

// --- Test Cases ---

console.log("=== Symbolic Differentiation Tests ===\n");

// Basic
test("x + 1");
test("x * x");

// Power Rule (Rewritten)
test("x ** 2");

// Chain Rule
test("exp(x * x)");
test("ln(x + 1) / x");

// Complex
test("x ** x");


## Imports and Setup

In [ ]:
import {
  createToken,
  Lexer,
  CstParser,
  IToken,
  ILexingResult,
  TokenType,
  CstNode,
} from "chevrotain";

# Symbolic Differentiation

In this notebook, we implement a system for **symbolic differentiation**. We construct a pipeline that parses an arithmetic expression string into an Abstract Syntax Tree (AST), applies differentiation rules symbolically, and converts the result back into a string.

## Architectural Overview

The pipeline consists of four stages:

1.  **Lexing:** Converts the input string into a stream of tokens (e.g., `EXP`, `X`, `NUMBER`).
2.  **Parsing:** Constructs a Concrete Syntax Tree (CST) using an **LL(k)** recursive-descent parser (Chevrotain).
3.  **AST Transformation:** Converts the CST into a simplified AST (`ExprNode`). This step handles operator precedence and associativity (especially the right-associativity of `**`).
4.  **Differentiation:** Recursively traverses the AST to compute the derivative $\frac{\mathrm{d}}{\mathrm{d}x}$.
5.  **Pretty Printing:** Converts the resulting AST back into a human-readable string, minimizing parentheses based on precedence rules.

## Grammar Specification

The grammar describes arithmetic expressions involving addition, subtraction, multiplication, division, exponentiation (`**`), the natural exponential function (`exp`), and the natural logarithm (`ln`).

To accommodate the **LL(k)** nature of our parser, recursive rules are expressed iteratively where possible:

$$
\begin{array}{lcl}
  \mathrm{expr}    & \rightarrow & \mathrm{product} \; \bigl( (\mathtt{'+'} \mid \mathtt{'-'}) \; \mathrm{product} \bigr)^* \\
  \mathrm{product} & \rightarrow & \mathrm{factor} \; \bigl( (\mathtt{'*'} \mid \mathtt{'/'}) \; \mathrm{factor} \bigr)^* \\
  \mathrm{factor}  & \rightarrow & \mathrm{base} \; \bigl( \mathtt{'**'} \; \mathrm{factor} \bigr)^? \quad \text{(Right-associative)} \\
  \mathrm{base}    & \rightarrow & \mathtt{exp} \; \mathtt{'('} \; \mathrm{expr} \;\mathtt{')'} \\
                   & \mid        & \mathtt{ln} \; \mathtt{'('} \; \mathrm{expr} \;\mathtt{')'} \\
                   & \mid        & \mathtt{'('} \; \mathrm{expr} \;\mathtt{')'} \\
                   & \mid        & \mathtt{NUMBER} \\
                   & \mid        & \mathtt{'x'}
\end{array}
$$

## 1. Specification of the Scanner

The scanner transforms the input into a sequence of tokens. We define specific tokens for mathematical functions and operators.

**Token Definitions:**

| Token Name | Pattern | Description |
| :--- | :--- | :--- |
| `NUMBER` | `0\|[1-9][0-9]*` | Natural numbers (integers) |
| `EXP` | `exp` | Exponential function $e^x$ |
| `LN` | `ln` | Natural logarithm $\ln(x)$ |
| `POWER` | `**` | Exponentiation operator |
| `X` | `x` | The variable to differentiate by |
| `Operators` | `+`, `-`, `*`, `/` | Basic arithmetic |
| `Parentheses`| `(`, `)` | Grouping |
| `WhiteSpace` | `[ \t\n\r]+` | Ignored characters |

In [ ]:
const NumberTok: TokenType = createToken({
  name: "NUMBER",
  pattern: /0|[1-9][0-9]*/,
});

In [ ]:
const ExpTok: TokenType = createToken({ name: "EXP", pattern: /exp/ });

In [ ]:
const LnTok: TokenType = createToken({ name: "LN", pattern: /ln/ });

Below, we need to escape the meta charater `*`.

In [ ]:
const PowerTok: TokenType = createToken({ name: "POWER", pattern: /\*\*/ });

In [ ]:
const Plus: TokenType = createToken({ name: "Plus", pattern: /\+/ });
const Minus: TokenType = createToken({ name: "Minus", pattern: /-/ });
const Mul: TokenType = createToken({ name: "Mul", pattern: /\*/ });
const Div: TokenType = createToken({ name: "Div", pattern: /\// });
const LParen: TokenType = createToken({ name: "LParen", pattern: /\(/ });
const RParen: TokenType = createToken({ name: "RParen", pattern: /\)/ });
const XTok: TokenType = createToken({ name: "X", pattern: /x/ });

Blanks and tabulators are ignored.

In [ ]:
const WhiteSpace: TokenType = createToken({
  name: "WhiteSpace",
  pattern: /[ \t\n\r]+/,
  group: Lexer.SKIPPED,
});

In [ ]:
const allTokens: TokenType[] = [
  WhiteSpace,
  NumberTok,
  ExpTok,
  LnTok,
  PowerTok,
  Plus,
  Minus,
  Mul,
  Div,
  LParen,
  RParen,
  XTok,
];

We generate the lexer.

In [ ]:
const DifferLexer: Lexer = new Lexer(allTokens);

### Helper Function `tokenizeDiff`

Wraps the lexer execution for easier debugging.

**Input:**
* `input`: The source string ($\texttt{string}$).

**Output:**
* A list of token images ($\texttt{string[]}$).

**Error Handling:**
Throws an error if the lexer encounters invalid characters.

In [ ]:
function tokenizeDiff(input: string): string[] {
  const lexRes: ILexingResult = DifferLexer.tokenize(input);
  if (lexRes.errors.length > 0) {
    throw new Error(`Lexing error: ${lexRes.errors[0].message}`);
  }
  return lexRes.tokens.map((t: IToken): string => t.image);
}

Example:

In [ ]:
console.log(tokenizeDiff("ln(x ** x) + exp(x * x)"));

## 2. Specification of the Parser & AST Types

We implement the parser by extending `CstParser`. The grammar rules defined above are mapped to methods (`expr`, `product`, `factor`, `base`).

### AST Type Definitions

Before implementing the visitor, we define the structure of our **Abstract Syntax Tree (AST)**. We use a recursive type definition `ExprNode`.

**Type `ExprNode`:**
An `ExprNode` is one of:
* `number`: A constant value ($c \in \mathbb{R}$).
* `"x"`: The variable symbol.
* `[UnaryOp, ExprNode]`: A unary operation, e.g., `["ln", x]`.
* `[BinaryOp, ExprNode, ExprNode]`: A binary operation, e.g., `["+", 2, x]`.

**Inputs/Outputs:**
The parser consumes the token vector and produces a **CST**.

In [ ]:
type BinaryOp = "+" | "-" | "*" | "/" | "**";
type UnaryOp = "ln" | "exp";

type ExprNode =
  | number
  | "x"
  | [UnaryOp, ExprNode]
  | [BinaryOp, ExprNode, ExprNode];

class DifferParser extends CstParser {
  public expr!: (idx?: number) => CstNode;
  public product!: (idx?: number) => CstNode;
  public factor!: (idx?: number) => CstNode;
  public base!: (idx?: number) => CstNode;

  constructor() {
    super(allTokens, { maxLookahead: 2 });
    const $ = this;

    // expr : product (('+' | '-') product)*
    $.RULE("expr", () => {
      $.SUBRULE($.product);
      $.MANY(() => {
        $.OR([
          {
            ALT: () => {
              $.CONSUME(Plus);
              $.SUBRULE2($.product);
            },
          },
          {
            ALT: () => {
              $.CONSUME(Minus);
              $.SUBRULE3($.product);
            },
          },
        ]);
      });
    });

    // product : factor (('*' | '/') factor)*
    $.RULE("product", () => {
      $.SUBRULE($.factor);
      $.MANY(() => {
        $.OR([
          {
            ALT: () => {
              $.CONSUME(Mul);
              $.SUBRULE2($.factor);
            },
          },
          {
            ALT: () => {
              $.CONSUME(Div);
              $.SUBRULE3($.factor);
            },
          },
        ]);
      });
    });

    // factor : base '**' factor | base
    $.RULE("factor", () => {
      $.SUBRULE($.base);
      $.OPTION(() => {
        $.CONSUME(PowerTok);
        $.SUBRULE2($.factor);
      });
    });

    // base : exp '(' expr ')' | ln '(' expr ')' | '(' expr ')' | NUMBER | 'x'
    $.RULE("base", () => {
      $.OR([
        {
          ALT: () => {
            $.CONSUME(ExpTok);
            $.CONSUME(LParen);
            $.SUBRULE($.expr);
            $.CONSUME(RParen);
          },
        },
        {
          ALT: () => {
            $.CONSUME(LnTok);
            $.CONSUME2(LParen);
            $.SUBRULE2($.expr);
            $.CONSUME2(RParen);
          },
        },
        {
          ALT: () => {
            $.CONSUME3(LParen);
            $.SUBRULE3($.expr);
            $.CONSUME3(RParen);
          },
        },
        { ALT: () => $.CONSUME(NumberTok) },
        { ALT: () => $.CONSUME(XTok) },
      ]);
    });

    this.performSelfAnalysis();
  }
}

const parser: DifferParser = new DifferParser();
const BaseCstVisitor = parser.getBaseCstVisitorConstructor();

## 3. CST to AST Transformation (Visitor)

The `ToASTVisitor` converts the generic CST into our typed `ExprNode`.

### Algorithm: Associativity and Precedence

1.  **Left-Associativity (`expr`, `product`):**
    The grammar defines `expr` as a sequence of products separated by `+` or `-`. To ensure strict left-to-right evaluation (e.g., $a - b + c \rightarrow (a-b)+c$), we use a specific algorithm:
    * **Collect**: Retrieve all operator tokens (`Plus`, `Minus`) into a single list.
    * **Sort**: Sort these tokens by their `startOffset` to respect the original textual order.
    * **Fold**: Iterate through the operands and the sorted operators, constructing binary nodes from left to right.

2.  **Right-Associativity (`factor`):**
    Exponentiation is right-associative ($a^{b^c} \rightarrow a^{(b^c)}$).
    * **Recursion**: The grammar rule `factor -> base (** factor)?` naturally handles this via recursion. If a `POWER` token exists, the visitor recursively visits the right-hand side `factor`, making it the right child of the operation.

**Input:**
A CST Node Context.

**Output:**
An `ExprNode` representing the logical structure of the expression.

In [ ]:
class ToASTVisitor extends BaseCstVisitor {
  constructor() {
    super();
    (this as unknown as { validateVisitor(): void }).validateVisitor();
  }

  // expr : product (('+' | '-') product)*
  public expr(ctx: {
    product: CstNode[];
    Plus?: IToken[];
    Minus?: IToken[];
  }): ExprNode {
    let node: ExprNode = this.visit(ctx.product[0]) as ExprNode;

    if (ctx.product.length > 1) {
      const plusTokens = ctx.Plus || [];
      const minusTokens = ctx.Minus || [];
      const allOps = [...plusTokens, ...minusTokens].sort(
        (a, b) => a.startOffset - b.startOffset
      );

      for (let i = 1; i < ctx.product.length; i++) {
        const rightNode = this.visit(ctx.product[i]) as ExprNode;
        const operator = allOps[i - 1];

        if (operator.tokenType.name === "Plus") {
          node = ["+", node, rightNode];
        } else {
          node = ["-", node, rightNode];
        }
      }
    }

    return node;
  }

  // product : factor (('*' | '/') factor)*
  public product(ctx: {
    factor: CstNode[];
    Mul?: IToken[];
    Div?: IToken[];
  }): ExprNode {
    let node: ExprNode = this.visit(ctx.factor[0]) as ExprNode;

    if (ctx.factor.length > 1) {
      const mulTokens = ctx.Mul || [];
      const divTokens = ctx.Div || [];
      const allOps = [...mulTokens, ...divTokens].sort(
        (a, b) => a.startOffset - b.startOffset
      );

      for (let i = 1; i < ctx.factor.length; i++) {
        const rightNode = this.visit(ctx.factor[i]) as ExprNode;
        const operator = allOps[i - 1];

        if (operator.tokenType.name === "Mul") {
          node = ["*", node, rightNode];
        } else {
          node = ["/", node, rightNode];
        }
      }
    }

    return node;
  }

  // factor : base '**' factor | base
  public factor(ctx: {
    base: CstNode[];
    POWER?: IToken[];
    factor?: CstNode[];
  }): ExprNode {
    const baseNode: ExprNode = this.visit(ctx.base[0]) as ExprNode;
    // Right-associative logic is handled correctly by recursion here
    if (ctx.POWER && ctx.factor) {
      const right: ExprNode = this.visit(ctx.factor[0]) as ExprNode;
      return ["**", baseNode, right];
    }
    return baseNode;
  }

  // base : exp '(' expr ')' | ln '(' expr ')' | '(' expr ')' | NUMBER | 'x'
  public base(ctx: {
    EXP?: IToken[];
    LN?: IToken[];
    expr?: CstNode[];
    LParen?: IToken[];
    NUMBER?: IToken[];
    X?: IToken[];
  }): ExprNode {
    if (ctx.EXP) {
      return ["exp", this.visit(ctx.expr![0]) as ExprNode];
    }
    if (ctx.LN) {
      return ["ln", this.visit(ctx.expr![0]) as ExprNode];
    }
    // Parentheses case: needs expr AND LParen
    if (ctx.expr && ctx.LParen) {
      return this.visit(ctx.expr[0]) as ExprNode;
    }
    if (ctx.NUMBER) {
      return parseInt(ctx.NUMBER[0].image, 10);
    }
    if (ctx.X) {
      return "x";
    }
    throw new Error("Unexpected base rule");
  }
}

const toAST: ToASTVisitor = new ToASTVisitor();

## 4. Parsing Interface

### Function `parseExpr`

Orchestrates the conversion from string to AST.

**Input:**
* `s`: Arithmetic expression string ($\texttt{string}$).

**Output:**
* An AST root node ($\texttt{ExprNode}$).

**Process:**
1.  Tokenize input.
2.  Parse tokens into CST using the `expr` rule.
3.  Visit CST to produce AST.

In [ ]:
function parseExpr(s: string): ExprNode {
  const lexRes: ILexingResult = DifferLexer.tokenize(s);
  if (lexRes.errors.length > 0) {
    throw new Error(`Lexing error: ${lexRes.errors[0].message}`);
  }

  parser.input = lexRes.tokens;
  const cst: CstNode = parser.expr();

  if (parser.errors.length > 0) {
    throw new Error(`Parse error: ${parser.errors[0].message}`);
  }

  return toAST.visit(cst) as ExprNode;
}

Example AST:

In [ ]:
console.log(parseExpr("ln(x ** x) + exp(x * x)"));

## 5. Symbolic Differentiation

### Function `diff`

Computes the symbolic derivative of an expression tree with respect to $x$.

**Input:**
* `e`: The expression tree ($\texttt{ExprNode}$) representing function $f(x)$.

**Output:**
* A new expression tree ($\texttt{ExprNode}$) representing $f'(x)$.

**Mathematical Algorithm:**

The function applies differentiation rules recursively:

1.  **Constants:** $\frac{\mathrm{d}}{\mathrm{d}x}(c) = 0$
2.  **Variable:** $\frac{\mathrm{d}}{\mathrm{d}x}(x) = 1$
3.  **Sum/Difference (Linearity):**
    $$\frac{\mathrm{d}}{\mathrm{d}x}(u \pm v) = u' \pm v'$$
4.  **Product Rule:**
    $$\frac{\mathrm{d}}{\mathrm{d}x}(u \cdot v) = u' \cdot v + u \cdot v'$$
5.  **Quotient Rule:**
    $$\frac{\mathrm{d}}{\mathrm{d}x}\left(\frac{u}{v}\right) = \frac{u' \cdot v - u \cdot v'}{v^2}$$
6.  **Chain Rule (Functions):**
    * $\frac{\mathrm{d}}{\mathrm{d}x}(\ln(u)) = \frac{u'}{u}$
    * $\frac{\mathrm{d}}{\mathrm{d}x}(\exp(u)) = u' \cdot \exp(u)$
7.  **Generalized Power Rule:**
    For $u^v$, we rewrite it as $\exp(v \cdot \ln(u))$ and apply the chain rule:
    $$\frac{\mathrm{d}}{\mathrm{d}x}(u^v) = \frac{\mathrm{d}}{\mathrm{d}x}\bigl(\exp(v \cdot \ln(u))\bigr)$$
    The implementation simply constructs the AST for $\exp(v \cdot \ln(u))$ and recursively calls `diff()` on it.

In [ ]:
function diff(e: ExprNode): ExprNode {
  if (typeof e === "number") return 0;
  if (e === "x") return 1;
  if (typeof e === "string") return 0;

  const [op, f, g] = e as [string, ExprNode, ExprNode];

  switch (op) {
    case "+":
      return ["+", diff(f), diff(g)];
    case "-":
      return ["-", diff(f), diff(g)];
    case "*":
      return ["+", ["*", diff(f), g], ["*", f, diff(g)]];
    case "/":
      return [
        "/",
        ["-", ["*", diff(f), g], ["*", f, diff(g)]],
        ["*", g, g],
      ];
    case "**":
      // f(x)**g(x) = exp(g(x)*ln(f(x))) → differentiate that
      return diff(["exp", ["*", g, ["ln", f]]]);
    case "ln":
      return ["/", diff(f), f];
    case "exp":
      return ["*", diff(f), e];
  }
  return 0;
}

## 6. Pretty Printing

### Function `toStringExpr`

Converts an AST back into a readable string representation. It attempts to produce minimal output by omitting unnecessary parentheses.

**Input:**
* `e`: An expression tree ($\texttt{ExprNode}$).

**Output:**
* A formatted string.

**Algorithm:**
The function assigns a **precedence level** to each operator:
* `+`, `-`: Level 1
* `*`, `/`: Level 2
* `**`: Level 3
* Atomic: Level 4

When printing a binary operation, it checks the precedence of the children. Parentheses are added only if the child operation has a lower precedence than the parent operation (or equal, depending on associativity).

In [ ]:
function precedence(op: string): number {
  const Precedences: Record<string, number> = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3,
  };
  return Precedences[op] ?? 4;
}

function precedenceOp(expr: ExprNode): number {
  if (Array.isArray(expr)) return precedence(expr[0]);
  return 4;
}

function toStringExpr(e: ExprNode): string {
  if (typeof e === "number" || typeof e === "string") {
    return String(e);
  }

  if (Array.isArray(e) && e.length === 2) {
    return e[0] + "(" + toStringExpr(e[1]) + ")";
  }

  if (e[0] === "+") {
    return toStringExpr(e[1]) + " + " + toStringExpr(e[2]);
  }

  if (e[0] === "-") {
    const lhs: string = toStringExpr(e[1]);
    const rhs: string =
      precedenceOp(e[2]) === 1
        ? "(" + toStringExpr(e[2]) + ")"
        : toStringExpr(e[2]);
    return lhs + " - " + rhs;
  }

  if (e[0] === "*") {
    const lhs: string =
      precedenceOp(e[1]) === 1
        ? "(" + toStringExpr(e[1]) + ")"
        : toStringExpr(e[1]);
    const rhs: string =
      precedenceOp(e[2]) === 1
        ? "(" + toStringExpr(e[2]) + ")"
        : toStringExpr(e[2]);
    return lhs + "*" + rhs;
  }

  if (e[0] === "/") {
    const lhs: string =
      precedenceOp(e[1]) === 1
        ? "(" + toStringExpr(e[1]) + ")"
        : toStringExpr(e[1]);
    const rhs: string =
      precedenceOp(e[2]) <= 2
        ? "(" + toStringExpr(e[2]) + ")"
        : toStringExpr(e[2]);
    return lhs + "/" + rhs;
  }

  if (e[0] === "**") {
    const lhs: string =
      precedenceOp(e[1]) <= 3
        ? "(" + toStringExpr(e[1]) + ")"
        : toStringExpr(e[1]);
    const rhs: string =
      precedenceOp(e[2]) <= 2
        ? "(" + toStringExpr(e[2]) + ")"
        : toStringExpr(e[2]);
    return lhs + "**" + rhs;
  }

  return "?";
}

## 7. Main Interface & Testing

### Function `diffString`

The high-level entry point for the user.

**Input:**
* `s`: Expression string.

**Output:**
* The derivative as a string.

**Process:**
$$\text{String} \xrightarrow{\text{parse}} \text{AST} \xrightarrow{\text{diff}} \text{AST'} \xrightarrow{\text{toString}} \text{String'}$$

In [ ]:
function diffString(s: string): string {
  const t: ExprNode = parseExpr(s);
  const d: ExprNode = diff(t);
  return toStringExpr(d);
}

function test(s: string): void {
  console.log(`d/dx ${s} = ${diffString(s)}`);
}

Example tests:

In [ ]:
test("x ** x");

In [ ]:
test("x * ln(x) / exp(x/x)");

unexpected error: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at updateContent (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:606:9)
    at Object.convert (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:257:9)
    at Object.execute (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\executor.js:138:38)
    at JupyterHandlerImpl.handleExecuteImpl (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\jupyter.js:25